# Understanding the MSI file
The Matrix Signal Information (MSI) file describes the information that is shown on the matrix signs above highways. This information can be imported in a csv file, where each line describes a change in MSI. We want to process this information, until we can deduce:
- For any time and any location on the highway (hectometer marker, inside the range), the available lanes and maximum speeds.

We will find this by first looking in the excel file "backwards over the highway", looking for the closest matrix sign. Then, we will search backwards in time for the latest update.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

In [11]:
# Define paths
datafolder = os.path.join(os.path.dirname(os.getcwd()), "data", "short_highway")
datafolder_msi = os.path.join(datafolder, "msi-export")
file1_path = os.path.join(datafolder_msi, "msi-export.csv")

In [12]:
# import file into pd.df
df = pd.read_csv(file1_path, low_memory=False)
df["time"] = pd.to_datetime(df["Datum en tijd beeldstandwijziging"])

In [21]:
def find_road_situation(hectometer, measure_time, MSI_df, direction="R", max_speed=100, num_lanes=6):
    lanedata = np.zeros(num_lanes)
    # Find closest location
    loc_df = MSI_df[MSI_df["DVK"] == direction]
    hm_points = loc_df.Hectometrering.unique()
    if direction == "R": 
        closest_measuring_location = max(hm_points[hm_points <= hectometer])
    else:
        closest_measuring_location = min(hm_points[hm_points >= hectometer])
    # Only look at the closest location
    loc_df = loc_df[loc_df["Hectometrering"] == closest_measuring_location]
    for lane_nr in range(1, num_lanes+1):
        # If there is no lane_nr in the closest location, then there is no lane. Set speed 0.
        lane_df = loc_df[loc_df["Rijstrook"] == lane_nr]
        if lane_df.empty:
            lanedata[lane_nr - 1] = 0
            continue
        # Find the latest update
        latest_update_time = max(lane_df[lane_df.time <= measure_time].time)
        beeldstand = lane_df[lane_df.time == latest_update_time]["Beeldstand"].values[0]
        # Update lanedata according to beeldstand
        if beeldstand in ["blank", "lane_closed_ahead merge_left", "lane_closed_ahead merge_right", "restriction_end"]:
            lanedata[lane_nr - 1] = max_speed
        elif beeldstand in ["lane_closed"]:
            lanedata[lane_nr - 1] = 0
        elif beeldstand.startswith("speedlimit"):
            lanedata[lane_nr - 1] = int(beeldstand.split(" ")[-1])
        else:
            raise KeyError(f"Beeldstand {beeldstand} is not known.")
    return(lanedata)



# Applying the function
We are now able to systematically find the road situation for all locations on a highway, although the code is not very efficient. Here is an example of how to use this function. We will also add this function to godunovfunctions.py.

In [24]:

measure_time = datetime.datetime.strptime("2023-10-16 15:00:00", "%Y-%m-%d %H:%M:%S")
for i in np.linspace(12, 17, 30):
    print(i, find_road_situation(i, measure_time, df, "L"))

12.0 [100. 100. 100. 100. 100.   0.]
12.172413793103448 [100. 100. 100.   0.   0.   0.]
12.344827586206897 [100. 100. 100.   0.   0.   0.]
12.517241379310345 [100. 100. 100.   0.   0.   0.]
12.689655172413794 [100. 100. 100.   0.   0.   0.]
12.862068965517242 [100. 100. 100.   0.   0.   0.]
13.03448275862069 [100. 100. 100.   0.   0.   0.]
13.206896551724139 [100. 100. 100.   0.   0.   0.]
13.379310344827587 [100. 100. 100.   0.   0.   0.]
13.551724137931036 [100. 100. 100.   0.   0.   0.]
13.724137931034482 [100. 100. 100.   0.   0.   0.]
13.89655172413793 [100. 100. 100.   0.   0.   0.]
14.068965517241379 [100. 100. 100.   0.   0.   0.]
14.241379310344827 [100. 100. 100.   0.   0.   0.]
14.413793103448276 [100. 100. 100.   0.   0.   0.]
14.586206896551724 [100. 100. 100.   0.   0.   0.]
14.758620689655173 [100. 100. 100.   0.   0.   0.]
14.931034482758621 [100. 100. 100.   0.   0.   0.]
15.10344827586207 [100. 100. 100.   0.   0.   0.]
15.275862068965518 [100. 100. 100.   0.   0.   0

ValueError: min() arg is an empty sequence